In [1]:
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

In [2]:
db_link ="/home/mateusz/Desktop/Moje_repo/My-Data-Science-repository/Stock_data_project/mWIG40/WIG40/WIG40/WIG.db"
conn = sqlite3.connect(db_link)

df = pd.read_sql('SELECT * FROM WIG',
            conn)

dates = [datetime.strptime(x, "%d-%m-%Y") for x in df["date"]]
df["date"] = dates

In [3]:
df.loc[df["ticker"]==df.ticker.unique()[0]].set_index("date")

,ticker,open,MAX,MIN,close,volume,trade
date,,,,,,,
2022-01-14,MAGNA POLONIA SPOLKA AKCYJNA,3.170,3.175,3.055,3.170,31747.0,98868.0
2022-01-13,MAGNA POLONIA SPOLKA AKCYJNA,3.200,3.200,3.140,3.170,10047.0,31692.0
2022-01-12,MAGNA POLONIA SPOLKA AKCYJNA,3.180,3.190,3.115,3.190,15092.0,47669.0
2022-01-11,MAGNA POLONIA SPOLKA AKCYJNA,3.140,3.190,3.100,3.100,21381.0,66695.0
2022-01-10,MAGNA POLONIA SPOLKA AKCYJNA,3.250,3.270,3.100,3.190,36051.0,114582.0
...,...,...,...,...,...,...,...
2022-03-25,MAGNA POLONIA SPOLKA AKCYJNA,3.630,3.670,3.620,3.665,5922.0,21660.0
2022-03-24,MAGNA POLONIA SPOLKA AKCYJNA,3.680,3.680,3.570,3.670,32730.0,119597.0
2022-03-23,MAGNA POLONIA SPOLKA AKCYJNA,3.625,3.750,3.625,3.680,34746.0,128319.0


In [6]:
x = df.loc[df['ticker'] == "MAGNA POLONIA SPOLKA AKCYJNA"].sort_values(by="date")
x

,date,ticker,open,MAX,MIN,close,volume,trade
19759,2016-01-04,MAGNA POLONIA SPOLKA AKCYJNA,1.750,1.830,1.75,1.830,1109.0,2016.0
19748,2016-01-05,MAGNA POLONIA SPOLKA AKCYJNA,1.750,1.820,1.75,1.820,4516.0,8195.0
19736,2016-01-07,MAGNA POLONIA SPOLKA AKCYJNA,1.750,1.820,1.75,1.820,11.0,20.0
19725,2016-01-08,MAGNA POLONIA SPOLKA AKCYJNA,1.750,1.830,1.75,1.830,137.0,249.0
19714,2016-01-11,MAGNA POLONIA SPOLKA AKCYJNA,1.750,1.830,1.75,1.830,3727.0,6523.0
...,...,...,...,...,...,...,...,...
466753,2022-05-09,MAGNA POLONIA SPOLKA AKCYJNA,3.310,3.465,3.31,3.400,9212.0,30863.0
466752,2022-05-10,MAGNA POLONIA SPOLKA AKCYJNA,3.435,3.435,3.28,3.280,5485.0,18280.0
466751,2022-05-11,MAGNA POLONIA SPOLKA AKCYJNA,3.390,3.395,3.25,3.365,10274.0,34275.0
466750,2022-05-12,MAGNA POLONIA SPOLKA AKCYJNA,3.270,3.375,3.12,3.295,10899.0,35159.0


In [8]:
periods = 14
ema = True
close_delta = x["close"].diff(1).dropna()
# make 2 series - up and down
up = close_delta.clip(lower=0)
down = -1*close_delta.clip(upper=0)
if ema == True:
    ma_up = up.ewm(com=periods-1, adjust=True, min_periods=periods).mean()
    ma_down = down.ewm(com=periods-1, adjust=True, min_periods=periods).mean()
else:
    #use simple moving avg
    ma_up = up.rolling(window=periods).mean()
    ma_down = down.rolling(window=periods).mean()
    
rsi = ma_up/ma_down
rsi = 100 - (100/(1+rsi))
x["rsi"] = rsi
x

,date,ticker,open,MAX,MIN,close,volume,trade,rsi
19759,2016-01-04,MAGNA POLONIA SPOLKA AKCYJNA,1.750,1.830,1.75,1.830,1109.0,2016.0,NaN
19748,2016-01-05,MAGNA POLONIA SPOLKA AKCYJNA,1.750,1.820,1.75,1.820,4516.0,8195.0,NaN
19736,2016-01-07,MAGNA POLONIA SPOLKA AKCYJNA,1.750,1.820,1.75,1.820,11.0,20.0,NaN
19725,2016-01-08,MAGNA POLONIA SPOLKA AKCYJNA,1.750,1.830,1.75,1.830,137.0,249.0,NaN
19714,2016-01-11,MAGNA POLONIA SPOLKA AKCYJNA,1.750,1.830,1.75,1.830,3727.0,6523.0,NaN
...,...,...,...,...,...,...,...,...,...
466753,2022-05-09,MAGNA POLONIA SPOLKA AKCYJNA,3.310,3.465,3.31,3.400,9212.0,30863.0,42.683185
466752,2022-05-10,MAGNA POLONIA SPOLKA AKCYJNA,3.435,3.435,3.28,3.280,5485.0,18280.0,39.384660
466751,2022-05-11,MAGNA POLONIA SPOLKA AKCYJNA,3.390,3.395,3.25,3.365,10274.0,34275.0,42.759027
466750,2022-05-12,MAGNA POLONIA SPOLKA AKCYJNA,3.270,3.375,3.12,3.295,10899.0,35159.0,40.747285


In [ ]:
df_list = []


In [10]:
tickers = df.ticker.unique()
tickers
for i in tickers:
    print(i)

MAGNA POLONIA SPOLKA AKCYJNA
BANK HANDLOWY W WARSZAWIE SPOLKA AKCYJNA
BENEFIT SYSTEMS SPOLKA AKCYJNA
ELEKTROCIEPLOWNIA BĘDZIN SPOLKA AKCYJNA
BUDIMEX SPOLKA AKCYJNA
BIG CHEESE STUDIO SPOLKA AKCYJNA
BOOMBIT SPOLKA AKCYJNA
BBI DEVELOPMENT SPOLKA AKCYJNA
BRITISH AUTOMOTIVE HOLDING SPOLKA AKCYJNA
BRAND 24 SPOLKA AKCYJNA
AIRWAY MEDIX SPOLKA AKCYJNA
AUGA GROUP AB
GRUPA AZOTY SPOLKA AKCYJNA
ATLANTIS SE
ATREM SPOLKA AKCYJNA
ATLANTA POLAND SPOLKA AKCYJNA
ZUE SPOLKA AKCYJNA
ZAKLAD BUDOWY MASZYN ZREMB-CHOJNICE SPOLKA AKCYJNA
ZESPOL ELEKTROWNI PĄTNOW-ADAMOW-KONIN SPOLKA AKCYJNA
ZAMET INDUSTRY SPOLKA AKCYJNA
WAWEL SPOLKA AKCYJNA
WARIMPEX FINANZ- UND BETEILIGUNGS AG
X-TRADE BROKERS DOM MAKLERSKI SPOLKA AKCYJNA
BUMECH SPOLKA AKCYJNA
BIO PLANET SPOLKĄ AKCYJNA
BIOTON SPOLKA AKCYJNA
XTPL SPOLKA AKCYJNA
WITTCHEN SPOLKA AKCYJNA
WIKANA SPOLKA AKCYJNA
WIELTON SPOLKA AKCYJNA
WOJAS SPOLKA AKCYJNA
WIRTUALNA POLSKA HOLDING SPOLKA AKCYJNA
WASKO SPOLKA AKCYJNA
VIVID GAMES SPOLKA AKCYJNA
VISTAL GDYNIA SPOLKA AKCYJN